In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import torchvision.models as models

In [3]:
net = models.resnet18(pretrained=False)

In [4]:
net.fc = nn.Linear(512,5)

In [5]:
#net = Model()
net = net.cuda()

In [6]:
torch.cuda.is_available()

True

In [7]:
reading = np.load('reading_np.npy')
phn = np.load('phone_np.npy')
drinking = np.load('drinking_np.npy')
cleaning = np.load('cleaning_np.npy')
#walking = np.load('walking_np.npy')
cutting = np.load('cutting_np.npy')

X = np.concatenate((reading, phn, drinking, cleaning, cutting), axis = 0)
# X = np.concatenate((reading, phn, drinking), axis = 0)

y_reading = np.full(50,0)
y_phn = np.full(50,1)
y_drink = np.full(50,2)
y_cleaning = np.full(50,3)
#y_walking = np.full(50,4)
y_cutting = np.full(50,4)

Y = np.concatenate((y_reading, y_phn, y_drink, y_cleaning, y_cutting), axis = 0)
# Y = np.concatenate((y_reading, y_phn, y_drink), axis = 0)

In [8]:
# Y = np.load('labels.npy')
# Y = Y - 1
# X = np.load('x.npy')
# X = np.array(X, dtype=float)

train_x_lh = X[:,:,4:8,:]
train_x_rh = X[:,:,8:12,:]
train_x_ll = X[:,:,12:16,:]
train_x_rl = X[:,:,16:20,:]
train_x_t = X[:,:,0:4,:]

final_x = np.concatenate((train_x_lh, train_x_rh), axis=2)
final_x = np.concatenate((final_x, train_x_t), axis=2)
final_x = np.concatenate((final_x, train_x_rl), axis=2)
final_x = np.concatenate((final_x, train_x_ll), axis=2)

X = final_x

In [9]:
X.shape

(250, 100, 20, 3)

In [10]:
#### Preprocessing to make the image.
min_ = X.min(axis=(0,1,2))
max_ = X.max(axis = (0,1,2))
X_normed = 255*(X - min_)/(max_ - min_)


In [47]:
min_

array([-1.08857226, -0.96385038,  1.34686899])

In [48]:
max_

array([1.50460005, 1.26765823, 4.08341026])

In [11]:
X.shape


(250, 100, 20, 3)

In [12]:
X_normed = np.pad(X_normed,[(0,0),(62,62),(102,102),(0,0)], 'constant')
X = X_normed

In [13]:
# X_normed = np.pad(X_normed,[(0,0),(37,37),(102,102),(0,0)], 'constant')
# X = X_normed

In [14]:
X_normed.shape

(250, 224, 224, 3)

In [15]:
N = Y.shape[0]

In [16]:
s = np.arange(X.shape[0])
np.random.shuffle(s)
shuffled_x = X[s]
shuffled_y = Y[s]


In [17]:
train_x = shuffled_x[0:int(0.8*N)]
train_y = shuffled_y[0:int(0.8*N)]
test_x = shuffled_x[int(0.8*N):]
test_y = shuffled_y[int(0.8*N):]

In [18]:
train_x = Variable(torch.from_numpy(train_x)).float().cuda()
train_x = train_x.permute(0,3,1,2).contiguous()
test_x = Variable(torch.from_numpy(test_x)).float().cuda()
test_x = test_x.permute(0,3,1,2).contiguous()
labels = Variable(torch.from_numpy(train_y)).cuda()
test_y = Variable(torch.from_numpy(test_y)).cuda()

In [19]:
test_x.shape

torch.Size([50, 3, 224, 224])

In [20]:
train_x.shape

torch.Size([200, 3, 224, 224])

In [21]:
BATCH_SIZE = 32
num_epochs = 20
num_batches = train_x.shape[0]/BATCH_SIZE
num_test_batches = test_x.shape[0]/BATCH_SIZE
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)

In [22]:
for epoch in range(num_epochs):
    running_loss = 0
    test_running_loss = 0
    for batch in range(int(num_test_batches)):
        batch_x = test_x[BATCH_SIZE*batch:BATCH_SIZE*(batch+1)]
        out = net(batch_x)
        loss = criterion(out,test_y[BATCH_SIZE*batch:BATCH_SIZE*(batch+1)].cuda())
        test_running_loss += loss.data[0]
        test_running_loss = test_running_loss/num_test_batches
        #test_loss_list.append(test_running_loss)
        
    for batch in range(int(num_batches)):
        optimizer.zero_grad()
        batch_x = train_x[BATCH_SIZE*batch:BATCH_SIZE*(batch+1)]
        out = net(batch_x)
        loss = criterion(out,labels[BATCH_SIZE*batch:BATCH_SIZE*(batch+1)])
        running_loss += loss.data[0]
        loss.backward()
        optimizer.step()
    
    print ('Epoch [%d/%d], Loss: %.4f, Test Loss: %.4f' %(epoch+1, num_epochs, running_loss/num_batches, test_running_loss/num_test_batches))

Epoch [1/20], Loss: 1.2395, Test Loss: 0.8042
Epoch [2/20], Loss: 0.8188, Test Loss: 0.2906
Epoch [3/20], Loss: 0.6451, Test Loss: 0.2548
Epoch [4/20], Loss: 0.5232, Test Loss: 0.2283
Epoch [5/20], Loss: 0.4499, Test Loss: 0.2124
Epoch [6/20], Loss: 0.4742, Test Loss: 0.2487
Epoch [7/20], Loss: 0.3134, Test Loss: 0.2501
Epoch [8/20], Loss: 0.2977, Test Loss: 0.1931
Epoch [9/20], Loss: 0.2098, Test Loss: 0.2910
Epoch [10/20], Loss: 0.1704, Test Loss: 0.2183
Epoch [11/20], Loss: 0.1588, Test Loss: 0.2862
Epoch [12/20], Loss: 0.1659, Test Loss: 0.2348
Epoch [13/20], Loss: 0.2560, Test Loss: 0.3047
Epoch [14/20], Loss: 0.2254, Test Loss: 0.2356
Epoch [15/20], Loss: 0.2507, Test Loss: 0.2533
Epoch [16/20], Loss: 0.1922, Test Loss: 0.2702
Epoch [17/20], Loss: 0.1064, Test Loss: 0.2426
Epoch [18/20], Loss: 0.1271, Test Loss: 0.2331
Epoch [19/20], Loss: 0.1193, Test Loss: 0.2805
Epoch [20/20], Loss: 0.1113, Test Loss: 0.1910


In [23]:
def pred(x):
    out = net(x)
    return np.argmax(out.data.cpu().numpy(), axis=1)
def accuracy(x,labels):
    N = labels.shape[0]
    predicted = pred(x)
    i = 0
    for yhat,y in zip(predicted,labels):
        
        if yhat == y[0]:
            i += 1
    return i

In [24]:
predicted = pred(test_x)

In [25]:
predicted.shape

(50,)

In [26]:
n_correct = np.sum(predicted == test_y.data.cpu().numpy())

In [27]:
n_correct


43

In [37]:
n_correct *100 / 50

86.0

In [38]:
predicted

array([4, 1, 2, 1, 0, 1, 1, 1, 3, 1, 4, 0, 4, 0, 2, 3, 0, 2, 1, 1, 1, 2,
       3, 4, 2, 0, 2, 2, 2, 1, 0, 0, 3, 0, 3, 4, 4, 0, 1, 4, 2, 0, 3, 4,
       1, 1, 3, 3, 3, 4])

In [39]:
test_y.data.cpu().numpy()

array([4, 1, 2, 1, 0, 1, 1, 0, 3, 1, 4, 0, 4, 0, 2, 3, 0, 0, 1, 1, 1, 2,
       3, 0, 2, 0, 2, 2, 0, 1, 0, 0, 3, 0, 1, 2, 4, 0, 1, 4, 0, 0, 3, 4,
       1, 1, 3, 3, 3, 4])

In [40]:
op = np.load('outputs_np.npy')
op = 255*(op - min_)/(max_ - min_)
op = np.pad(op,[(0,0),(62,62),(102,102),(0,0)], 'constant')

In [41]:
op = Variable(torch.from_numpy(op)).cuda()

In [42]:
op = op.repeat(32,1,1,1)

In [43]:
op.shape

torch.Size([32, 224, 224, 3])

In [44]:
op = op.permute(0,3,1,2).contiguous()
op = op.float()
pred(op)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [46]:
op = np.load('test_data_np.npy')
min_op = op.min(axis = (0,1,2))
max_op = op.max(axis = (0,1,2))

op = 255*(op - min_)/(min_op - max_op)
op = np.pad(op,[(0,0),(62,62),(102,102),(0,0)], 'constant')
op = Variable(torch.from_numpy(op)).cuda()
op = op.repeat(4,1,1,1)
op = op.permute(0,3,1,2).contiguous()
op = op.float()
pred(op)

array([3, 3, 4, 3, 3, 1, 0, 0, 3, 3, 4, 3, 3, 1, 0, 0, 3, 3, 4, 3, 3, 1,
       0, 0, 3, 3, 4, 3, 3, 1, 0, 0])